# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2026-01-07 21:07:22] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2026-01-07 21:07:22] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2026-01-07 21:07:22] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2026-01-07 21:07:25] INFO server_args.py:1615: Attention backend not specified. Use fa3 backend by default.


[2026-01-07 21:07:25] INFO server_args.py:2502: Set soft_watchdog_timeout since in CI


...

Capturing batches (bs=88 avail_mem=75.24 GB):  20%|██        | 4/20 [00:00<00:01, 13.61it/s]

Capturing batches (bs=80 avail_mem=75.23 GB):  30%|███       | 6/20 [00:00<00:00, 15.55it/s]

Capturing batches (bs=40 avail_mem=74.71 GB):  55%|█████▌    | 11/20 [00:00<00:00, 14.68it/s]

Capturing batches (bs=8 avail_mem=74.69 GB):  70%|███████   | 14/20 [00:01<00:00, 17.86it/s] 

Capturing batches (bs=1 avail_mem=74.64 GB): 100%|██████████| 20/20 [00:01<00:00, 16.94it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Phebe and I'm a Graphic Designer and Art Instructor. I have been a graphic designer for almost 5 years, working with all different types of clients, from small businesses to Fortune 500 companies. I create design materials, and teach classes to help my clients achieve their design vision. I have a BFA in Graphic Design from the University of Texas and a Masters in Graphic Design from the University of Texas.
I've been teaching Graphic Design to art students and teachers for the past 7 years. I believe that Graphic Design is an art form that requires students to learn about and understand the technicalities and the design process
Prompt: The president of the United States is
Generated text:  a very important person, with important responsibilities. It is reported that President Xi Jinping recently visited the Beidaihe, the "labor" in the Red Square. From this, we can see that the president of the United States is ____.
A: A member of the Commun

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name], and I'm a [job title] at [company name]. I'm excited to meet you and learn more about you. What can you tell me about yourself? I'm a [insert a short description of your job or profession]. I enjoy [insert a short description of your hobbies or interests]. I'm always looking for new challenges and opportunities to grow and learn. What's your favorite hobby or activity? I'm always looking for new ways to challenge myself and expand my knowledge. What's your favorite book or movie? I'm always looking for new ways to challenge myself and expand my knowledge. What's your favorite color

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, also known as the City of Light, and is located in the south of the country. It is the largest city in Europe and the second-largest city in the world by population. Paris is known for its rich history, art, and culture, and is home to many famous landmarks such 

Generated text:  likely to be characterized by several key trends:

1. Increased integration with human intelligence: AI is likely to become more integrated with human intelligence, allowing machines to learn from and adapt to human behavior and decision-making processes.

2. Greater emphasis on ethical considerations: As AI becomes more prevalent in various industries, there will be a greater emphasis on ethical considerations and the responsible use of AI.

3. Development of more advanced AI systems: AI systems will continue to become more advanced and capable, with the ability to learn from more complex data and make more informed decisions.

4. Increased focus on privacy and security: As AI systems become more integrated into our



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  Sarah, and I'm a dedicated and intelligent speaker. I love learning new things and am always eager to expand my knowledge. I enjoy working with groups of people to generate ideas and come up with solutions to problems. I have a knack for storytelling and can captivate groups and lead discussions with my engaging and clear communication style. I'm always on the lookout for new ways to learn and grow, and I'm excited to get started with you! #self-introduction #Speaker #Learner #Innovator #Talkative #Creative #Talker #Presentation #Decision-maker #Knowledge-Builder #Soft-Minded #Collaborative

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, the city that has been a historical and cultural hub for over 2, 500 years and has been described as a "city of love" and a "city of light" by both poets and artists a

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 __

________

.

 I

...

 greater

 need

 to

 ensure

 that

In [6]:
llm.shutdown()